In [1]:
# import the necessary packages
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dropout
from keras.layers.core import Dense
from keras import backend as K

Using TensorFlow backend.


In [3]:
class SmallVGGNet:
	@staticmethod
	def build(width, height, depth, classes):
        
		# initialize the model along with the input shape to be
		# "channels last" and the channels dimension itself
        
		model = Sequential()
		inputShape = (height, width, depth)
		chanDim = -1
        
		# if we are using "channels first", update the input shape
		# and channels dimension
        
		if K.image_data_format() == "channels_first":
			inputShape = (depth, height, width)
			chanDim = 1
            
		# CONV => RELU => POOL layer set
		model.add(Conv2D(32, (3, 3), padding="same",input_shape=inputShape))
		model.add(Activation("relu"))
		model.add(BatchNormalization(axis=chanDim))
		model.add(MaxPooling2D(pool_size=(2, 2)))
		model.add(Dropout(0.25))
        
		# (CONV => RELU) * 2 => POOL layer set
		model.add(Conv2D(64, (3, 3), padding="same"))
		model.add(Activation("relu"))
		model.add(BatchNormalization(axis=chanDim))
		model.add(Conv2D(64, (3, 3), padding="same"))
		model.add(Activation("relu"))
		model.add(BatchNormalization(axis=chanDim))
		model.add(MaxPooling2D(pool_size=(2, 2)))
		model.add(Dropout(0.25))
        
		# (CONV => RELU) * 3 => POOL layer set
		model.add(Conv2D(128, (3, 3), padding="same"))
		model.add(Activation("relu"))
		model.add(BatchNormalization(axis=chanDim))
		model.add(Conv2D(128, (3, 3), padding="same"))
		model.add(Activation("relu"))
		model.add(BatchNormalization(axis=chanDim))
		model.add(Conv2D(128, (3, 3), padding="same"))
		model.add(Activation("relu"))
		model.add(BatchNormalization(axis=chanDim))
		model.add(MaxPooling2D(pool_size=(2, 2)))
		model.add(Dropout(0.25))
        
		# first (and only) set of FC => RELU layers
		model.add(Flatten())
		model.add(Dense(512))
		model.add(Activation("relu"))
		model.add(BatchNormalization())
		model.add(Dropout(0.5))
        
		# softmax classifier
		model.add(Dense(classes))
		model.add(Activation("softmax"))
        
		# return the constructed network architecture
		return model

In [5]:
# set the matplotlib backend so figures can be saved in the background
import matplotlib
matplotlib.use("Agg")
# import the necessary packages

from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from keras.preprocessing.image import ImageDataGenerator #data augmentation
from keras.optimizers import SGD
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import random
import pickle
import cv2
import os

In [7]:
# initialize the data and labels
print("[INFO] loading images...")
data = []
labels = []
# grab the image paths and randomly shuffle them

imagePaths = sorted(list(paths.list_images("animals/")))
random.seed(40)
random.shuffle(imagePaths)

# loop over the input images
for imagePath in imagePaths:
	# load the image, resize it to 64x64 pixels (the required input
	# spatial dimensions of SmallVGGNet), and store the image in the
	# data list
	image = cv2.imread(imagePath)
	image = cv2.resize(image, (64, 64))
	data.append(image)
    
	# extract the class label from the image path and update the
	# labels list
    
	label = imagePath.split(os.path.sep)[-2]
	labels.append(label)
# scale the raw pixel intensities to the range [0, 1]
data = np.array(data, dtype="float") / 255.0
labels = np.array(labels)

[INFO] loading images...


In [8]:
# partition the data into training and testing splits using 75% of
# the data for training and the remaining 25% for testing
(trainX, testX, trainY, testY) = train_test_split(data,labels, test_size=0.25, random_state=42)

lb = LabelBinarizer()
trainY = lb.fit_transform(trainY)
testY = lb.transform(testY)

In [9]:
# construct the image generator for data augmentation
aug = ImageDataGenerator(rotation_range=30, width_shift_range=0.1,height_shift_range=0.1, shear_range=0.2, zoom_range=0.2,
	horizontal_flip=True, fill_mode="nearest")

# initialize our VGG-like Convolutional Neural Network

model = SmallVGGNet.build(width=64, height=64, depth=3,classes=len(lb.classes_))

In [10]:
# initialize our initial learning rate, # of epochs to train for,
# and batch size

INIT_LR = 0.01
EPOCHS = 75
BS = 32

# initialize the model and optimizer (you'll want to use
# binary_crossentropy for 2-class classification)
print("[INFO] training network...")
opt = SGD(lr=INIT_LR, decay=INIT_LR / EPOCHS) #learning rate decay
model.compile(loss="categorical_crossentropy", optimizer=opt,metrics=["accuracy"])

# train the network
H = model.fit_generator(aug.flow(trainX, trainY, batch_size=BS),validation_data=(testX, testY), steps_per_epoch=len(trainX) // BS,epochs=EPOCHS)

[INFO] training network...
Epoch 1/75
70/70 [==============================] - 105s 2s/step - loss: 1.4305 - accuracy: 0.5261 - val_loss: 1.9043 - val_accuracy: 0.3213
Epoch 2/75
70/70 [==============================] - 90s 1s/step - loss: 1.0578 - accuracy: 0.5712 - val_loss: 1.7358 - val_accuracy: 0.3200
Epoch 3/75
70/70 [==============================] - 93s 1s/step - loss: 0.9341 - accuracy: 0.6177 - val_loss: 2.3543 - val_accuracy: 0.3240
Epoch 4/75
70/70 [==============================] - 93s 1s/step - loss: 0.8781 - accuracy: 0.6100 - val_loss: 2.3610 - val_accuracy: 0.3800
Epoch 5/75
70/70 [==============================] - 91s 1s/step - loss: 0.7947 - accuracy: 0.6289 - val_loss: 1.9627 - val_accuracy: 0.4133
Epoch 6/75
70/70 [==============================] - 92s 1s/step - loss: 0.7668 - accuracy: 0.6289 - val_loss: 1.5321 - val_accuracy: 0.4227
Epoch 7/75
70/70 [==============================] - 93s 1s/step - loss: 0.7279 - accuracy: 0.6605 - val_loss: 0.7348 - val_accuracy:

Epoch 59/75
70/70 [==============================] - 110s 2s/step - loss: 0.4725 - accuracy: 0.7864 - val_loss: 1.0343 - val_accuracy: 0.6387
Epoch 60/75
70/70 [==============================] - 113s 2s/step - loss: 0.4638 - accuracy: 0.7902 - val_loss: 0.5136 - val_accuracy: 0.7653
Epoch 61/75
70/70 [==============================] - 112s 2s/step - loss: 0.4568 - accuracy: 0.7885 - val_loss: 0.5466 - val_accuracy: 0.7613
Epoch 62/75
70/70 [==============================] - 113s 2s/step - loss: 0.4535 - accuracy: 0.7931 - val_loss: 0.5936 - val_accuracy: 0.7453
Epoch 63/75
70/70 [==============================] - 113s 2s/step - loss: 0.4725 - accuracy: 0.7750 - val_loss: 0.4923 - val_accuracy: 0.7773
Epoch 64/75
70/70 [==============================] - 113s 2s/step - loss: 0.4622 - accuracy: 0.7827 - val_loss: 0.5785 - val_accuracy: 0.7507
Epoch 65/75
70/70 [==============================] - 113s 2s/step - loss: 0.4716 - accuracy: 0.7759 - val_loss: 0.5454 - val_accuracy: 0.7533
Epoch 

In [12]:
# evaluate the network
print("[INFO] evaluating network...")
predictions = model.predict(testX, batch_size=32)
print(classification_report(testY.argmax(axis=1),predictions.argmax(axis=1), target_names=lb.classes_))

# plot the training loss and accuracy
N = np.arange(0, EPOCHS)
plt.style.use("ggplot")
plt.figure()
plt.plot(N, H.history["loss"], label="train_loss")
plt.plot(N, H.history["val_loss"], label="val_loss")
plt.plot(N, H.history["accuracy"], label="train_acc")
plt.plot(N, H.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy (SmallVGGNet)")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend()
plt.savefig("plot.jpg")

# save the model and label binarizer to disk
print("[INFO] serializing network and label binarizer...")
model.save("smallvggmodel")
f = open("labelvgg", "wb")
f.write(pickle.dumps(lb))
f.close()

[INFO] evaluating network...
               precision    recall  f1-score   support

 animals/cats       0.66      0.73      0.69       241
 animals/dogs       0.74      0.46      0.57       254
animals/panda       0.76      0.98      0.85       255

     accuracy                           0.72       750
    macro avg       0.72      0.72      0.70       750
 weighted avg       0.72      0.72      0.70       750

[INFO] serializing network and label binarizer...
